In [1]:
#importing essential libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters 
num_epochs = 5
batch_size = 32
learning_rate = 0.001

# Define data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a uniform size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Load dataset
dataset = ImageFolder(root='/kaggle/input/inaturalist2/inaturalist_12K/train', transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = ImageFolder(root='/kaggle/input/inaturalist2/inaturalist_12K/val', transform=transform)

# Create data loaders
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [2]:
def freeze_layers(model, freeze_percent=0.5):
    total_layers = len(list(model.parameters()))
    num_layers_to_freeze = int(total_layers * freeze_percent)
    layers_frozen = 0

    for name, param in model.named_parameters():
        if layers_frozen < num_layers_to_freeze:
            param.requires_grad = False
            layers_frozen += 1
        else:
            break
'''
# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)

# Verify if layers are frozen

for name, param in model.named_parameters():
    print(name, param.requires_grad)
    
print(model)'''

'\n# Load pre-trained ResNet50 model\nmodel = models.resnet50(pretrained=True)\nnum_features = model.fc.in_features\nmodel.fc = nn.Linear(num_features, 10)\n\n# Verify if layers are frozen\n\nfor name, param in model.named_parameters():\n    print(name, param.requires_grad)\n    \nprint(model)'

In [3]:
!pip install wandb

In [4]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [5]:
wandb.login(key='cd7a6c2259e8886dc269bbf6f0f9e55089d3beeb')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep test6',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'freezing' : {
            'values' : [0.7,0.8,0.9]
        }
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project='Deep_Learning_Assignment2_PartB')

Create sweep with ID: dbm5ipr4
Sweep URL: https://wandb.ai/prabhat-kumar/Deep_Learning_Assignment2_PartB/sweeps/dbm5ipr4


In [7]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init(entity = 'prabhat-kumar') as run:

        run_name="-frz"+str(wandb.config.freezing)
        wandb.run.name=run_name
#         actv = get_activation(wandb.config.activation)
        
        # Load pre-trained ResNet50 model
        model = models.resnet50(pretrained=True)
        
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)
        model = model.to(device)
        
        freeze_layers(model, freeze_percent=wandb.config.freezing)
        
        

        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(num_epochs):
            total_train = 0
            correct_train = 0
            running_loss = 0
            # Train the model
            model.train()
            for images, labels in train_loader:
                
                images = images.to(device)
                labels = labels.to(device)
        
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
        
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # Calculate the training accuracy
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                
            train_accuracy = 100*correct_train/total_train
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {train_accuracy:.2f}%')
            wandb.log({'train_loss': round(running_loss / len(train_loader), 2)})
            wandb.log({'train_accuracy': round(train_accuracy, 2)})

            #wandb.log({'train_loss':running_loss/len(train_loader)})
            #wandb.log({'train_accuracy':train_accuracy})
            
        
    
            # Validate the model
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in val_loader:
                    
                    images = images.to(device)
                    labels = labels.to(device)
            
                    # Forward pass
                    outputs = model(images)
                    _, predicted = torch.max(outputs, 1)
            
                    # Compute accuracy
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
        
                val_accuracy = 100 * correct / total
                print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')
                wandb.log({'val_accuracy':val_accuracy})
                wandb.log({'epoch':epoch+1})
    

        print('Finished Training')

wandb.agent(sweep_id, function=main,count=5) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: rnf3cykm with config:
wandb: 	freezing: 0.7
wandb: Currently logged in as: prabhat-kumar. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 166MB/s] 


Epoch [1/5], Training Accuracy: 57.04%
Epoch [1/5], Validation Accuracy: 63.10%
Epoch [2/5], Training Accuracy: 71.68%
Epoch [2/5], Validation Accuracy: 72.20%
Epoch [3/5], Training Accuracy: 80.51%
Epoch [3/5], Validation Accuracy: 74.30%
Epoch [4/5], Training Accuracy: 87.47%
Epoch [4/5], Validation Accuracy: 70.75%
Epoch [5/5], Training Accuracy: 91.19%
Epoch [5/5], Validation Accuracy: 71.70%
Finished Training


epoch,▁▃▅▆█
train_accuracy,▁▄▆▇█
train_loss,█▅▃▂▁
val_accuracy,▁▇█▆▆
epoch,5
train_accuracy,91.19
train_loss,0.27
val_accuracy,71.7


wandb: Agent Starting Run: o4ael4li with config:
wandb: 	freezing: 0.9
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

Epoch [1/5], Training Accuracy: 65.15%
Epoch [1/5], Validation Accuracy: 74.95%
Epoch [2/5], Training Accuracy: 77.20%
Epoch [2/5], Validation Accuracy: 75.50%
Epoch [3/5], Training Accuracy: 82.60%
Epoch [3/5], Validation Accuracy: 77.35%
Epoch [4/5], Training Accuracy: 87.52%
Epoch [4/5], Validation Accuracy: 74.25%
Epoch [5/5], Training Accuracy: 90.94%
Epoch [5/5], Validation Accuracy: 74.70%
Finished Training


epoch,▁▃▅▆█
train_accuracy,▁▄▆▇█
train_loss,█▅▃▂▁
val_accuracy,▃▄█▁▂
epoch,5
train_accuracy,90.94
train_loss,0.27
val_accuracy,74.7


wandb: Agent Starting Run: wrgov614 with config:
wandb: 	freezing: 0.9
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch [1/5], Training Accuracy: 65.16%
Epoch [1/5], Validation Accuracy: 74.55%
Epoch [2/5], Training Accuracy: 76.81%
Epoch [2/5], Validation Accuracy: 73.70%
Epoch [3/5], Training Accuracy: 83.06%
Epoch [3/5], Validation Accuracy: 73.95%
Epoch [4/5], Training Accuracy: 87.00%
Epoch [4/5], Validation Accuracy: 76.60%
Epoch [5/5], Training Accuracy: 91.36%
Epoch [5/5], Validation Accuracy: 78.25%
Finished Training


epoch,▁▃▅▆█
train_accuracy,▁▄▆▇█
train_loss,█▅▃▂▁
val_accuracy,▂▁▁▅█
epoch,5
train_accuracy,91.36
train_loss,0.26
val_accuracy,78.25
